In [3]:
!pip install tqdm

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [259]:
from tqdm import tqdm_notebook as tqdm 
import numpy as np
from collections import deque, defaultdict, OrderedDict
import timeit
import pandas as pd
import random

In [48]:
df = pd.read_csv('smalltrace.csv', sep='\t')
df.columns = ['no','timestamp','pid','pname','blockNo', 'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']
blocktrace = df['blockNo'].tolist()

In [108]:
bt = list(np.random.randint(10, size=(50)))

In [185]:

def belady_opt(blocktrace, frame):
    
    OPT = defaultdict(deque)
    latestIndex = set()
    
    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block].append(i)    

    hit, miss = 0, 0
    C = set()
    for block in tqdm(blocktrace, desc="OPT", leave=False):
        OPT[block].popleft()
        if block in C:
            if len(latestIndex) == frame:
                for i in latestIndex:
                    if blocktrace[i] == block:
                        latestIndex.remove(i)
                        break
            print('here')
            #latestIndex.remove(i for i in latestIndex if blocktrace[i] in C)
            try:
                latestIndex.add(OPT[block][0])
            except:
                pass
            hit+=1
           
        else:
            miss+=1
            if len(C) == frame:
                #eviction
                if len(latestIndex) == 0:
                    C.remove(random.sample(C,1)[0])
                else:
                    maxIndex = max(latestIndex)
                    evictBlock = blocktrace[maxIndex]
                    latestIndex.remove(maxIndex)
                    C.remove(evictBlock)
            
            #insertion
            C.add(block)
            try:
                latestIndex.add(OPT[block][0])
            except:
                pass
                

  
    hitrate = hit / (hit + miss)
    return hitrate

In [51]:
maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            return cached_block            
    for cached_block in C:
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    return maxAccessBlock


IndentationError: unexpected indent (<ipython-input-51-ef9e17c77414>, line 2)

In [7]:
l = []


In [212]:
def getFurthestAccessBlock(C, OPT):
    maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            return cached_block            
    for cached_block in C:
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    return maxAccessBlock

def belady_opt(blocktrace, frame):
    OPT = defaultdict(deque)

    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block].append(i)    

    #print ("created OPT dictionary")    
    hit, miss = 0, 0
    C = set()
    

    for block in tqdm(blocktrace, desc="OPT"):

        if block in C:
            #OPT[block] = OPT[block][1:]
            OPT[block].popleft()
            hit+=1
            #print('hit' + str(block))
            #print(OPT)
        else:
            #print('miss' + str(block))
            miss+=1
            if len(C) == frame:
                fblock = getFurthestAccessBlock(C, OPT)
                assert(fblock != -1)
                C.remove(fblock)
            C.add(block)
            #OPT[block] = OPT[block][1:]
            #print(OPT)
            OPT[block].popleft()
            #print(block)
    #print ("hit count" + str(hit_count))
        print(C)
    #print ("miss count" + str(miss_count))
    hitrate = hit / (hit + miss)
    #print(hitrate)
    return hitrate

In [71]:
l2  = belady_opt(blocktrace, 500)

In [261]:
d = OrderedDict()

In [262]:
d['jay'] = 3

In [263]:
d

OrderedDict([('jay', 3)])

In [93]:
l3 = list(set(l2) - set(l))

In [94]:
len(l3)

5876

In [95]:
len(l) - len(l2)

171

In [88]:
for i in l:
    print(i)
    break

(1, 488846200)


In [91]:
(1, 488846200) not in l3

True

In [92]:
{1,2,3,4} - {4,5,6,7}

{1, 2, 3}

In [169]:
belady_opt(bt, 3) #new

4
0
5
2
6
9
7
4
1
0
3
9
7
8
2
0
1
3
7
6
4
3
2
0
4
9
5
6



0.44

In [137]:
len('''4
0
5
2
6
9
7
4
1
0
3
9
7
8
2
0
1
3
7
6
4
3
2
0
4
9
5
6''')


55

In [186]:
belady_opt(bt, 3) #old

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


KeyError: 2

In [135]:
len('''4
0
5
2
5
6
9
7
9
1
6
0
3
9
4
9
4
8
2
0
1
2
3
7
6
7
8
3
0
7
4
9
5
6''')

67

In [162]:
set3 = {1,2,3,4,5,65,7,8,9,9}

In [163]:
set3

{1, 2, 3, 4, 5, 7, 8, 9, 65}

In [165]:
random.sample(set3,1)

[4]

In [172]:
se = set()


In [174]:
print(se)

set()


In [175]:
for i in se:
    print(i)

In [232]:
def belady_opttt2(blocktrace, frame):
    
    C = set()
    hit, miss = 0, 0
    
    indexer = defaultdict(deque)
    futureIndex = set()
    
    #build index
    for i, block in enumerate(blocktrace):
        indexer[block].append(i)
    
    
    for block in blocktrace:
        
        if block in C:
            try:
                futureIndex.remove(indexer[block][0])
                indexer[block].popleft()
                futureIndex.add(indexer[block][0])
            except:
                pass
            hit += 1
            #print(block, '\t', C, '\t', futureIndex)

        else:
            miss += 1
            if len(C) == frame:
                if len(futureIndex) < frame:
                    evictBlock = random.choice([i for i in C if len(indexer[i]) == 0]) 
                    C.remove(evictBlock)
                else:
                    C.remove(blocktrace[max(futureIndex)])
                    futureIndex.remove(max(futureIndex))

            C.add(block)
            indexer[block].popleft()
            try:
                futureIndex.add(indexer[block][0])
            except:
                pass
            #print(block, '\t', C, '\t', futureIndex, '\t-----------------------------')
    hitrate = hit / (hit + miss)
    return hitrate

In [243]:
!pip install sortedcontainers

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [244]:
import sortedcontainers

In [304]:
def belady_opttt3(blocktrace, frame):
    
    C = set()
    hit, miss = 0, 0
    
    indexer = defaultdict(deque)
    futureIndex = SortedList()
    
    #build index
    for i, block in enumerate(blocktrace):
        indexer[block].append(i)
    
    
    for block in blocktrace:
        
        if block in C:
            try:
                futureIndex.remove(indexer[block][0])
                indexer[block].popleft()
                futureIndex.add(indexer[block][0])
            except:
                pass
            hit += 1
            #print(block, '\t', C, '\t', futureIndex)

        else:
            miss += 1
            if len(C) == frame:
                if len(futureIndex) < frame:
                    evictBlock = random.choice([i for i in C if len(indexer[i]) == 0]) 
                    C.remove(evictBlock)
                else:
                    C.remove(blocktrace[futureIndex.pop()])

            C.add(block)
            indexer[block].popleft()
            try:
                futureIndex.add(indexer[block][0])
            except:
                pass
            #print(block, '\t', C, '\t', futureIndex, '\t-----------------------------')
    hitrate = hit / (hit + miss)
    return hitrate

In [305]:
belady_opttt3(bt,3)

0.44

In [306]:
def belady_opttt4(blocktrace, frame):
    
    C = set()
    hit, miss = 0, 0
    
    indexer = defaultdict(deque)
    futureIndex = OrderedDict()
    
    #build index
    for i, block in enumerate(blocktrace):
        indexer[block].append(i)
    
    
    for block in blocktrace:
        
        if block in C:
            try:
                futureIndex.move_to_end(block)
                futureIndex.popitem()
                indexer[block].popleft()
                futureIndex[block] = indexer[block][0]
                futureIndex = OrderedDict(sorted(futureIndex.items(), key=lambda t: t[1]))
            except:
                pass
            hit += 1
            #print(block, '\t', C, '\t', futureIndex)

        else:
            miss += 1
            if len(C) == frame:
                if len(futureIndex) != frame:
                    evictBlock = random.choice([i for i in C if len(indexer[i]) == 0]) 
                    C.remove(evictBlock)
                else:
                    C.remove(blocktrace[futureIndex.popitem()[1]])

            C.add(block)
            indexer[block].popleft()
            try:
                futureIndex[block] = indexer[block][0]
                futureIndex = OrderedDict(sorted(futureIndex.items(), key=lambda t: t[1]))
            except:
                pass
            #print(block, '\t', C, '\t', futureIndex, '\t-----------------------------')
    hitrate = hit / (hit + miss)
    return hitrate

In [307]:
belady_opttt4(bt, 3)

0.44

In [301]:
hit

17

In [302]:
miss

33

In [303]:
miss2

30

In [290]:
belady_opttt(bt,3)

0.44

In [225]:
def belady_opttt(blocktrace, frame):
    
    C = set()
    hit, miss = 0, 0
    
    indexer = defaultdict(deque)
    futureIndex = deque()
    
    #build index
    for i, block in enumerate(blocktrace):
        indexer[block].append(i)
    
    
    for block in blocktrace:
        
        if block in C:
            try:
                futureIndex.remove(indexer[block][0])
                indexer[block].popleft()
                futureIndex.append(indexer[block][0])
            except:
                pass
            hit += 1
            #print(block, '\t', C, '\t', futureIndex)

        else:
            miss += 1
            if len(C) == frame:
                if len(futureIndex) < frame:
                    evictBlock = random.choice([i for i in C if len(indexer[i]) == 0]) 
                    C.remove(evictBlock)
                else:
                    C.remove(blocktrace[max(futureIndex)])
                    futureIndex.remove(max(futureIndex))

            C.add(block)
            indexer[block].popleft()
            try:
                futureIndex.append(indexer[block][0])
            except:
                pass
            #print(block, '\t', C, '\t', futureIndex, '\t-----------------------------')
    hitrate = hit / (hit + miss)
    return hitrate

In [189]:
try:
    print("jaaysa")
    print(fdfdfd)
    print(ffsfs)
    print("fdfdfd")
except:
    pass

jaaysa


In [226]:
belady_opttt(bt, 3)

0.44

In [310]:
setup = '''

from tqdm import tqdm_notebook as tqdm 
import numpy as np
from collections import deque, defaultdict, OrderedDict
import timeit
import pandas as pd
import random
from sortedcontainers import SortedList

df = pd.read_csv('smalltrace.csv', sep='\t')
df.columns = ['no','timestamp','pid','pname','blockNo', 'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']
blocktrace = df['blockNo'].tolist()


def getFurthestAccessBlock(C, OPT):
    maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            return cached_block            
    for cached_block in C:
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    return maxAccessBlock

def belady_opt(blocktrace, frame):
    OPT = defaultdict(deque)

    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block].append(i)    

    #print ("created OPT dictionary")    
    hit, miss = 0, 0
    C = set()
    

    for block in tqdm(blocktrace, desc="OPT"):

        if block in C:
            #OPT[block] = OPT[block][1:]
            OPT[block].popleft()
            hit+=1
            #print('hit' + str(block))
            #print(OPT)
        else:
            #print('miss' + str(block))
            miss+=1
            if len(C) == frame:
                fblock = getFurthestAccessBlock(C, OPT)
                assert(fblock != -1)
                C.remove(fblock)
            C.add(block)
            #OPT[block] = OPT[block][1:]
            #print(OPT)
            OPT[block].popleft()
            #print(block)
    #print ("hit count" + str(hit_count))
        #print(C)
    #print ("miss count" + str(miss_count))
    hitrate = hit / (hit + miss)
    #print(hitrate)
    return hitrate


def belady_opttt(blocktrace, frame):
    
    C = set()
    hit, miss = 0, 0
    
    indexer = defaultdict(deque)
    futureIndex = deque()
    
    #build index
    for i, block in enumerate(blocktrace):
        indexer[block].append(i)
    
    
    for block in blocktrace:
        
        if block in C:
            try:
                futureIndex.remove(indexer[block][0])
                indexer[block].popleft()
                futureIndex.append(indexer[block][0])
            except:
                pass
            hit += 1
            #print(block, '\t', C, '\t', futureIndex)

        else:
            miss += 1
            if len(C) == frame:
                if len(futureIndex) < frame:
                    evictBlock = random.choice([i for i in C if len(indexer[i]) == 0]) 
                    C.remove(evictBlock)
                else:
                    C.remove(blocktrace[max(futureIndex)])
                    futureIndex.remove(max(futureIndex))

            C.add(block)
            indexer[block].popleft()
            try:
                futureIndex.append(indexer[block][0])
            except:
                pass
            #print(block, '\t', C, '\t', futureIndex, '\t-----------------------------')
    hitrate = hit / (hit + miss)
    return hitrate

def belady_opttt2(blocktrace, frame):
    
    C = set()
    hit, miss = 0, 0
    
    indexer = defaultdict(deque)
    futureIndex = set()
    
    #build index
    for i, block in enumerate(blocktrace):
        indexer[block].append(i)
    
    
    for block in blocktrace:
        
        if block in C:
            try:
                futureIndex.remove(indexer[block][0])
                indexer[block].popleft()
                futureIndex.add(indexer[block][0])
            except:
                pass
            hit += 1
            #print(block, '\t', C, '\t', futureIndex)

        else:
            miss += 1
            if len(C) == frame:
                if len(futureIndex) < frame:
                    evictBlock = random.choice([i for i in C if len(indexer[i]) == 0]) 
                    C.remove(evictBlock)
                else:
                    C.remove(blocktrace[max(futureIndex)])
                    futureIndex.remove(max(futureIndex))

            C.add(block)
            indexer[block].popleft()
            try:
                futureIndex.add(indexer[block][0])
            except:
                pass
            #print(block, '\t', C, '\t', futureIndex, '\t-----------------------------')
    hitrate = hit / (hit + miss)
    return hitrate

def belady_opttt3(blocktrace, frame):
    
    C = set()
    hit, miss = 0, 0
    
    indexer = defaultdict(deque)
    futureIndex = SortedList()
    
    #build index
    for i, block in enumerate(blocktrace):
        indexer[block].append(i)
    
    
    for block in blocktrace:
        
        if block in C:
            try:
                futureIndex.remove(indexer[block][0])
                indexer[block].popleft()
                futureIndex.add(indexer[block][0])
            except:
                pass
            hit += 1
            #print(block, '\t', C, '\t', futureIndex)

        else:
            miss += 1
            if len(C) == frame:
                if len(futureIndex) < frame:
                    evictBlock = random.choice([i for i in C if len(indexer[i]) == 0]) 
                    C.remove(evictBlock)
                else:
                    C.remove(blocktrace[futureIndex.pop()])

            C.add(block)
            indexer[block].popleft()
            try:
                futureIndex.add(indexer[block][0])
            except:
                pass
            #print(block, '\t', C, '\t', futureIndex, '\t-----------------------------')
    hitrate = hit / (hit + miss)
    return hitrate

def belady_opttt4(blocktrace, frame):
    
    C = set()
    hit, miss = 0, 0
    
    indexer = defaultdict(deque)
    futureIndex = OrderedDict()
    
    #build index
    for i, block in enumerate(blocktrace):
        indexer[block].append(i)
    
    
    for block in blocktrace:
        
        if block in C:
            try:
                futureIndex.move_to_end(block)
                futureIndex.popitem()
                indexer[block].popleft()
                futureIndex[block] = indexer[block][0]
                futureIndex = OrderedDict(sorted(futureIndex.items(), key=lambda t: t[1]))
            except:
                pass
            hit += 1
            #print(block, '\t', C, '\t', futureIndex)

        else:
            miss += 1
            if len(C) == frame:
                if len(futureIndex) != frame:
                    evictBlock = random.choice([i for i in C if len(indexer[i]) == 0]) 
                    C.remove(evictBlock)
                else:
                    C.remove(blocktrace[futureIndex.popitem()[1]])

            C.add(block)
            indexer[block].popleft()
            try:
                futureIndex[block] = indexer[block][0]
                futureIndex = OrderedDict(sorted(futureIndex.items(), key=lambda t: t[1]))
            except:
                pass
            #print(block, '\t', C, '\t', futureIndex, '\t-----------------------------')
    hitrate = hit / (hit + miss)
    return hitrate

'''

In [236]:
timeit.timeit(setup=setup, stmt="belady_opt(blocktrace, 500)", number=1)

0.05817105556101865

In [231]:
timeit.timeit(setup=setup, stmt="belady_opttt(blocktrace, 500)", number=1)

0.43214681446663405

In [235]:
timeit.timeit(setup=setup, stmt="belady_opttt2(blocktrace, 500)", number=1)

0.4280097498192248

In [245]:
from sortedcontainers import SortedList

In [257]:
timeit.timeit(setup=setup, stmt="belady_opttt3(blocktrace, 500)", number=1)

0.4278945818439297

In [311]:
timeit.timeit(setup=setup, stmt="belady_opttt4(blocktrace, 500)", number=1)

0.43020884862198727

In [264]:
d

OrderedDict([('jay', 3)])

In [265]:
d['mittal'] = 4

In [266]:
d

OrderedDict([('jay', 3), ('mittal', 4)])

In [276]:
d['k'] = [1,2,3]

In [278]:
d

OrderedDict([('jay', 3), ('mittal', 4), ('k', [1, 2, 3])])

In [277]:
OrderedDict(sorted(d.items(), key=lambda t: t[1]))

TypeError: '<' not supported between instances of 'list' and 'int'

In [284]:
d.popitem()[0]

'mittal'

In [279]:
len(d)

3